In [1]:
import os
import torch
from transformers import SwinForImageClassification

c:\adam_kacmar\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
def validate(model, val_loader, criterion):
    model.eval()
    total_loss = 0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images).logits
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    return total_loss / len(val_loader), accuracy

In [6]:
models_dir = "models"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

if not os.path.exists(models_dir):
    print(f"The directory '{models_dir}' does not exist.")
else:
    print(f"Directory '{models_dir}' exists.")


criterion = torch.nn.CrossEntropyLoss()

results = []

Directory 'models' exists.


In [8]:
for epoch in range(10, 201, 10):
    model_path = os.path.join(models_dir, f"model_epoch_{epoch}.pth")
    
    model = SwinForImageClassification.from_pretrained(
        "microsoft/swinv2-base-patch4-window8-256", 
        num_labels=4,
        ignore_mismatched_sizes=True
    )
    
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)

    val_loss, val_accuracy = validate(model, val_loader, criterion, device)
    results.append({"epoch": epoch, "val_loss": val_loss, "val_accuracy": val_accuracy})
    print(f"Epoch {epoch}: Validation Loss = {val_loss:.4f}, Validation Accuracy = {val_accuracy:.2f}%")


You are using a model of type swinv2 to instantiate a model of type swin. This is not supported for all configurations of models and can yield errors.
Some weights of SwinForImageClassification were not initialized from the model checkpoint at microsoft/swinv2-base-patch4-window8-256 and are newly initialized: ['swin.embeddings.norm.bias', 'swin.embeddings.norm.weight', 'swin.embeddings.patch_embeddings.projection.bias', 'swin.embeddings.patch_embeddings.projection.weight', 'swin.encoder.layers.0.blocks.0.attention.output.dense.bias', 'swin.encoder.layers.0.blocks.0.attention.output.dense.weight', 'swin.encoder.layers.0.blocks.0.attention.self.key.bias', 'swin.encoder.layers.0.blocks.0.attention.self.key.weight', 'swin.encoder.layers.0.blocks.0.attention.self.query.bias', 'swin.encoder.layers.0.blocks.0.attention.self.query.weight', 'swin.encoder.layers.0.blocks.0.attention.self.relative_position_bias_table', 'swin.encoder.layers.0.blocks.0.attention.self.relative_position_index', 'swi

NameError: name 'val_loader' is not defined